In [ ]:
# Verification Proccess
# 1. Get CT source points from vessel extraction
# 2. Transform CT points given registration solution
# 3. Create combined image
# 4. Verify in imageviewer

In [ ]:
import os
import sys
import numpy as np
import itk
from itk import TubeTK as ttk
from itkwidgets import view
import math
import functions as func

In [ ]:
# Directory names
pipelineDir = "./Results/"

In [ ]:
# patient 2
tube_file = "./Data/Vessels_02/CT-Lungs-Vessels_02.tre"
tube_out_file = "./Results/CT-Lungs-Vessels_02_transformed.tre"
recon_file_3D = "./Data/TomoReconstruction_02/TR_02_02.nii"
ct_file = "./Data/CT_02/CT_02_04.nii"
solution_output_filename = "regSolution_02.txt"
combined_image_filename = "combined_02_untransformed.mha"
x_init = [0, -55, 10, 0, 0, 0]
img = itk.imread(ct_file)
spacing = img.GetSpacing()
spacing = spacing[0]

In [ ]:
# Change spacing in tomoRecon volume to correct spacing, write corrected image
img = itk.imread(recon_file_3D)
img.SetSpacing((0.194, 0.194, 3))
img.Update()

In [ ]:
# patient 2
itk.imwrite(img, "./Data/TomoReconstruction_02/TR_02_02.mha")
img_3D = "./Data/TomoReconstruction_02/TR_02_02.mha"

In [ ]:
solution = [0, 0, 0, 0, 0, 0]

In [ ]:
# Get tubes from vessel segmentation .tre file
size = itk.size(itk.imread(recon_file_3D))
PixelType = itk.F
Dimension = 3
ImageType = itk.Image[PixelType, Dimension]
TubeFileReaderType = itk.SpatialObjectReader[Dimension]
tubeFileReader = TubeFileReaderType.New()
tubeFileReader.SetFileName(tube_file)
tubeFileReader.Update()
tubes = tubeFileReader.GetGroup()

print("Number of objects = ", tubes.GetNumberOfChildren())
sobj = tubes.GetChildren(0)

# Transform vessel segmentation points given registration solution
for tube_num in range(len(sobj)):
    tube = itk.down_cast(sobj[tube_num])
    tube_points = tube.GetPoints()
    num_points = len(tube_points)
    print("Transforming the", num_points, "points in tube", tube_num)
    for point_num in range(num_points):
        x = tube_points[point_num].GetPositionInObjectSpace()
        new_point = [x[0], x[2], x[1]]
        new_point = func.TransformOnePointAllParameters(x_init, new_point)
        new_point = [((new_point[0]/(spacing))+(size[0]/2)*0.194), 
                     ((new_point[1]/(spacing))+(size[1]/2)*0.194), 
                     (((new_point[2]/0.6))*0.7079646+(size[2]/2)*3)*-1]
        tube_points[point_num].SetPositionInObjectSpace(new_point)

In [ ]:
# Save transformed points
tubesWriter = itk.SpatialObjectWriter[Dimension].New()
tubesWriter.SetInput(tubes)
tubesWriter.SetFileName(tube_out_file)
tubesWriter.Update()

In [ ]:
# Read transformed tubes, visualize tubes with tomosynthesis reconstruction volume
Dimension = 3

reader = itk.SpatialObjectReader[Dimension].New()
reader.SetFileName(tube_out_file)
reader.Update()
tubes = reader.GetGroup()

In [ ]:
view(image=img, point_sets=tubes)

In [ ]:
# Read template image
TemplateImageType = itk.Image[PixelType, Dimension]
TemplateImageReaderType = itk.ImageFileReader[TemplateImageType]
    
templateImageReader = TemplateImageReaderType.New()
templateImageReader.SetFileName(img_3D)
templateImageReader.Update()

templateImage = templateImageReader.GetOutput()

In [ ]:
# Create templated points
TubesToImageFilterType = ttk.ConvertTubesToImage[TemplateImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetTemplateImage(templateImageReader.GetOutput())
tubesToImageFilter.SetInput(tubes)
tubesToImageFilter.Update()

outputImage = tubesToImageFilter.GetOutput()

In [ ]:
view(outputImage)

In [ ]:
# Create combined image
TTKImageMathType = ttk.ImageMath[ImageType]

imMath = TTKImageMathType.New(Input = outputImage)
imMath.AddImages(templateImage, 2048, 1)
combinedImage = imMath.GetOutput()
WriterType = itk.ImageFileWriter[itk.Image[itk.F,3]]
writer = WriterType.New()
writer.SetFileName(pipelineDir+combined_image_filename)
writer.SetInput(combinedImage)
writer.Update()

In [ ]:
view(combinedImage)